# Simulation de la ligne HERMES avec PyOptiX

In [1]:
import ctypes
import sys, os
sys.path.append("..")
import numpy as np
from scipy.constants import h, c, eV, nano, milli, degree, pi
import pandas as pd
from classes import (Beamline, Source, OpticalElement, Diagram, RecordingMode, ToroidalMirror, 
                     PlaneHoloGrating, CylindricalMirror, ConicCylindricalMirror)
from exposed_functions import *
from ui_objects import show, plot_spd
from bokeh.io import push_notebook, output_notebook, show
from bokeh.plotting import figure
output_notebook()

Loading BokehJS ...

In [2]:
global optix
hc = h*c/nano/eV
try:
    test = optix
    print(test, "already initialized")
except NameError:
    optix = load_optix()
    print("OptiX library initialized")

intialzing SR library
OptiX library initialized


## Définition des optiques

In [3]:
Hermes = Beamline()
onduleur_BE = Source(element_type="GaussianSource", name="S_ONDUL_BE")
pupille = OpticalElement(element_type="PlaneFilm", name="pupille")
m1a = OpticalElement(element_type="PlaneMirror", name="M1A")
m1b = ToroidalMirror(name="M1B")
m1c = ToroidalMirror(name="M1C")
focalisation_horizontale = OpticalElement(element_type="PlaneFilm", name="foca")
f = OpticalElement(element_type="PlaneFilm", name="f")
reseau_450 = PlaneHoloGrating(name="Reseau_450")
reseau_600 = PlaneHoloGrating(name="Reseau_600")
reseau_200 = PlaneHoloGrating(name="Reseau_200")
reseau_300 = PlaneHoloGrating(name="Reseau_300")
reseau_MCA = PlaneHoloGrating(name="Reseau_1800Multi")
m2 = OpticalElement(element_type="PlaneMirror", name="plan")
m3 = ToroidalMirror(name="M3tor")
m4 = ToroidalMirror(name="M4")
plan_intermediaire = OpticalElement(element_type="PlaneFilm", name="planint")
fente = OpticalElement(element_type="PlaneFilm", name="Fente")
m5 = CylindricalMirror(name="M5CylConCave")
foc_intermediaire = OpticalElement(element_type="PlaneFilm", name="FocIntHo")
diagnostic = OpticalElement(element_type="PlaneFilm", name="Diag")
m6 = ConicCylindricalMirror(name="M6Cc")
m7 = ConicCylindricalMirror(name="M7Cc")
peem = OpticalElement(element_type="PlaneFilm", name="PEEM")
stxm = OpticalElement(element_type="PlaneFilm", name="STXM")

set_parameter (2807905774720, 'phi', <classes.Parameter object at 0x0000028DC3E4CBC0>) {} error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'
set_parameter (2807905774720, 'psi', <classes.Parameter object at 0x0000028DC3E4CBC0>) {} error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'
set_parameter (2807905774720, 'theta', <classes.Parameter object at 0x0000028DC3E4CBC0>) {} error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'
set_parameter (2807905774720, 'Dphi', <classes.Parameter object at 0x0000028DC3E4CBC0>) {} error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'
set_parameter (2807905774720, 'Dpsi', <classes.Parameter object at 0x0000028DC3E4CBC0>) {} error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'
set_parameter (280790

In [4]:
import classes
Hermes = classes.Beamline()
branches = [[m5, foc_intermediaire, m6, m7, peem],
            [m4, stxm]]
for ond, alias_ond in [(onduleur_BE, "BE"),(onduleur_BE, "HE")]:
    for reseau in [reseau_200, reseau_300, reseau_450, reseau_600, reseau_MCA]:
        for branch, branch_name in zip(branches, ["PEEM", "STXM"]):
            chain_name = f"{alias_ond}_{branch_name}_{reseau.name.split('_')[1]}"
            if alias_ond == "BE":
                second_m1 = m1b
            else:
                second_m1 = m1c
            Hermes.chains[chain_name] = [ond,
                                         pupille, m1a, second_m1, focalisation_horizontale, f, 
                                         reseau,
                                         plan_intermediaire, fente] + branch
print(Hermes.chains)
Hermes.active_chain = "BE_PEEM_200"
print(Hermes.active_chain)

Chaîne BE_PEEM_200:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_200 -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM -> 
Chaîne BE_STXM_200:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_200 -> planint -> Fente -> M4 -> STXM -> 
Chaîne BE_PEEM_300:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_300 -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM -> 
Chaîne BE_STXM_300:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_300 -> planint -> Fente -> M4 -> STXM -> 
Chaîne BE_PEEM_450:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_450 -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEEM -> 
Chaîne BE_STXM_450:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_450 -> planint -> Fente -> M4 -> STXM -> 
Chaîne BE_PEEM_600:
	S_ONDUL_BE -> pupille -> M1A -> M1B -> foca -> f -> Reseau_600 -> planint -> Fente -> M5CylConCave -> FocIntHo -> M6Cc -> M7Cc -> PEE

## Définition des paramètres des optiques

### Onduleur basse énergie

In [5]:
onduleur_BE.nrays = 5000
onduleur_BE.sigma_x = 1.74807e-4
onduleur_BE.sigma_y = 1.477385e-5
onduleur_BE.sigma_x_div = 3.17065e-3*degree # rad
onduleur_BE.sigma_y_div = 2.782436e-3*degree # rad
onduleur_BE.next = pupille
lambda_align = 6e-9
lambda_radiate = 6e-9

### Pupille

In [6]:
pupille.distance_from_previous = 20
pupille.recording_mode = RecordingMode.recording_output
pupille.next = m1a

### M1a 

In [7]:
m1a.distance_from_previous = 0.4783
m1a.theta = 2.5*degree # rad
m1a.phi = -90*degree # rad
m1a.next = m1b

### M1b 

In [8]:
m1b.distance_from_previous = 3.2096
m1b.theta = 2.5*degree # rad
m1b.phi = 180*degree # rad
m1b.minor_curvature = 1/1.802572 # m-1
m1b.major_curvature = 1/126.7433 # m-1
m1b.next = focalisation_horizontale

### M1c

In [9]:
m1b.distance_from_previous = 0.9791
m1b.theta = 1.2*degree # rad
m1b.phi = 180*degree # rad
m1b.minor_curvature = 1/0.8870043 # m-1
m1b.major_curvature = 1/229.5203 # m-1

### Plan de focalisation_horizontale

In [10]:
focalisation_horizontale.phi = -90*degree # rad
focalisation_horizontale.distance_from_previous = 3.2096
focalisation_horizontale.recording_mode = RecordingMode.recording_output
focalisation_horizontale.next = reseau_450

### Réseau 200 tpmm

In [11]:
reseau_200.line_density = 200/milli
reseau_200.inverse_distance1=0.1439452
reseau_200.inverse_distance2=0.1111111
reseau_200.elevation_angle1=np.arccos(0.7986355)

### Réseau 300 tpmm 

In [12]:
reseau_300.line_density = 300/milli
reseau_300.inverse_distance1=0.3247547
reseau_300.inverse_distance2=0.1111111
reseau_300.elevation_angle1=np.arccos(0.7986355)

### Reseau 450 tpmm

In [13]:
reseau_450.line_density = 450/milli
reseau_450.inverse_distance1=0.1761099
reseau_450.inverse_distance2=0.1111111
reseau_450.elevation_angle1=np.arccos(0.7986355)
reseau_450.next = m2

### Reseau 600 tpmm

In [14]:
reseau_600.line_density = 600/milli
reseau_600.inverse_distance1=0.2002547
reseau_600.inverse_distance2=0.1111111
reseau_600.elevation_angle1=np.arccos(0.7986355)

### Reseau MCA

In [15]:
reseau_MCA.line_density = 1800/milli
reseau_MCA.inverse_distance1=1e-9
reseau_MCA.inverse_distance2=1e-7
reseau_MCA.elevation_angle1=np.arccos(0.939693)
# Note : ce réseau n'est pas VLS

### M2


In [16]:
# l'angle et la distance sont calculés au moment de l'alignement du réseau
m2.phi = 180*degree # rad
m2.next = m3

### M3 

In [17]:
# la distance au M2-M3 dépend de l'énergie et est calculée à l'alignement
m3.phi = 90*degree # rad
m3.theta = (180-177.6)/2*degree # rad
m3.major_curvature = 1/83 # m-1
m3.minor_curvature = 1/0.1462124 # m-1
m3.next = plan_intermediaire

### Plan de focalisation intermédiaire 

In [18]:
plan_intermediaire.distance_from_previous = 0.2
plan_intermediaire.phi = -90*degree #rad
plan_intermediaire.next = fente

### Fente de sortie du mono

In [19]:
fente.distance_from_previous = 3.3
fente.recording_mode = RecordingMode.recording_output
fente.next = m5 # branche peem

### M5 

In [20]:
m5.distance_from_previous = 1
m5.phi = 90*degree
m5.curvature = 1/35 # m-1
m5.axis_angle = 0 # tangential cylinder 
m5.next = foc_intermediaire 

### Focalisation intermediaire 

In [21]:
foc_intermediaire.phi = -90*degree # rad
foc_intermediaire.distance_from_previous = 0.6187585
foc_intermediaire.next = diagnostic
foc_intermediaire.recording_mode = RecordingMode.recording_output

### Diagnostic faisceau

In [22]:
diagnostic.distance_from_previous = 7.7812242
diagnostic.recording_mode = RecordingMode.recording_output
diagnostic.next = m6

### M6 

In [23]:
m6.phi = 90*degree # rad
m6.theta = 1.75*degree # rad
m6.distance_from_previous = 0.6
m6.inverse_p = 1/(0.6 + 7.7812242) # m-1
m6.inverse_q = 1/(0.5 + 1.80) # m-1
m6.theta0 = 1.75*degree #rad
m6.next = m7

### M7

In [24]:
m7.distance_from_previous = 0.5
m7.phi = 90*degree # rad
m7.theta = 1.75*degree # rad
m7.inverse_p = 1/(0.6 + 7.7812242 + 0.5) # m-1
m7.inverse_q = 1/(1.80) # m-1
m7.theta0 = 1.75*degree #rad
m7.next = peem

### PEEM

In [25]:
peem.distance_from_previous = 1.80 # m
peem.recording_mode = RecordingMode.recording_output

## Définition des scripts d'alignement

### Onduleur

In [26]:
# *****     Données          **********
# dimension de la source électronique (sigmas en m)
#  SDM 169 X 6 µm ;    SDL 188 X 9 µm ;  SDC  321 x  8.1
Sh = 169e-6
Sv = 6e-6
# dimension de la source électronique (sigmas en rd)
#  SDM 27 X 5 µrd ;    SDL  30 x 4.5 µrd; SDC 14.6 x 4.5  µrd
Sph = 27e-6
Spv = 5e-6
# Longueur de l'onduleur (m)
Lond = 1.8
# Position (m) du centre de la S D au centre de l'onduleur
# Csd = dist SD / l'ond. utilisé comme origine (sens + vers l'aval)
# Cond = dist ond. / la SD utilisée comme origine (sens + vers l'aval)
Csd = 0
Cond = 1.849

# Facteur d'accord de l'onduleur   = augmentation de la divergence  en cas de desaccord
Ka = 1.4


# ************************************************
#      Elargissement du aux erreurs de pente
#
SSlV = 0
SSlH = 0
#SSlV = SlopeM1h * DistSM1 *  np.cos( M1dev /2.) + SlopeG1 * (DistSM1 + DistM1G ) )*2
# SSlH = $SlopeM1v *2 *  $DistSM1

#**************************************************


Lambda0 = lambda_radiate
print(f"Lambda = {lambda_radiate} ")
Su2 = Lambda0 * Lond / Ka/ (8 * pi**2) 
Spu2 = Lambda0 * Ka / (2 * Lond) 
print(f"SigmaU =  {np.sqrt(Su2)}  SigmaUp = {np.sqrt(Spu2)} ")
if  Cond == 0 :
    Zv = np.sqrt(Spv) / ( Spv**2 + Spu2)  * Csd 
    Zh = Sph**2 / ( Sph**2 + Spu2)  * Csd 
else :
    Zv = Spu2 / ( Spv**2 + Spu2)  * Cond 
    Zh = Spu2 / ( Sph**2 + Spu2)  * Cond 

print(f"Zv = {Zv}      Zh = {Zh} ")
# onduleur_BE.WaistZ = Zv
# onduleur_BE.WaistY = Zh
sqv2 = 1. / ( 1./Spv**2 + 1./Spu2 ) 
sqh2 = 1. / ( 1./Sph**2 + 1./Spu2 ) 
if  Cond == 0:
    SigmaV = np.sqrt(( Sv**2 + SSlV**2 + Su2 + Csd**2 *sqv2))
    SigmaH = np.sqrt(( Sh**2 + SSlH**2 + Su2 + Csd**2 *sqh2))
else:
    SigmaV = np.sqrt(( Sv**2 + SSlV**2 + Su2 + Cond**2 *sqv2))
    SigmaH = np.sqrt(( Sh**2 + SSlH**2 + Su2 + Cond**2 *sqh2))

print(f"sigmaY = {SigmaH}      sigmaZ = {SigmaV} ")
onduleur_BE.sigma_x = SigmaH
onduleur_BE.sigma_y =  SigmaV
SigmaPV = np.sqrt((Spv**2 + Spu2)) 
SigmaPH = np.sqrt((Sph**2 + Spu2)) 
print(f"sigmaYp = {SigmaPH}       sigmaZp = {SigmaPV}")
onduleur_BE.sigma_x_div = SigmaPH
onduleur_BE.sigma_y_div = SigmaPV

pupille.distance_from_previous += Cond - Csd # distance centre onduleur - centre source

Lambda = 6e-09 
SigmaU =  9.88446103441284e-06  SigmaUp = 4.8304589153964794e-05 
Zv = 1.829399293286219      Zh = 1.4088385762490476 
sigmaY = 0.00017480762475497748      sigmaZ = 1.4773846901636054e-05 
sigmaYp = 5.533835318595353e-05       sigmaZp = 4.856267428111155e-05


### Réseaux

In [27]:
cff = 0.6
omega = 0.72*degree # rad
delta_z = 15*milli # m
deviation = np.nan

if cff > 1:
    cff = 1/cff # selon definition
if focalisation_horizontale.next == reseau_MCA: # cas omega constant
    deviation = 2*np.arccos(lambda_align*reseau_MCA.line_density/2./np.sin(omega))
    alpha = (pi-deviation)/2 + omega
    beta =  (pi-deviation)/2 - omega
    reseau_MCA.theta = alpha
else: 
    reseau = focalisation_horizontale.next # cas cff constant
    nu = lambda_align*reseau.line_density
    k = 1-cff**2
    X = (np.sqrt(k**2 + nu**2*cff**2) - nu) / k
    alpha = np.arccos(X)
    beta = np.arccos(nu + X)
    deviation = pi - alpha - beta
    reseau.theta = alpha
    print(f"k={k}, nu={nu}, alpha={alpha}, beta={beta}")

    
m2.theta = (pi - deviation)/2
dist = delta_z/np.cos(deviation/2)*(1-np.sin(deviation/2)/2)
m2.distance_from_previous = dist
m3.distance_from_previous = 0.55 - dist*np.cos(deviation)

print(f"Alpha = {alpha/degree} deg,")
print(f"Beta = {beta/degree} deg")
print(f"Deviation = {deviation/degree} deg")
print(f"Distance reseau-M2 = {dist}")
print(f"Distance M2-M3 = {0.55 + dist*np.cos(deviation)}")

set_parameter (2807905774720, 'theta', <classes.Parameter object at 0x0000028DC7843BC0>) {} error b'invalid grating parameter in function setParameter of file D:\\Dennetiere\\optix\\src\\holo.cpp'
k=0.64, nu=0.0027, alpha=0.09185326919712176, beta=0.0550623166987319
Alpha = 5.262804659474084 deg,
Beta = 3.1548383570500538 deg
Deviation = 171.58235698347585 deg
Distance reseau-M2 = 0.1024669172245776
Distance M2-M3 = 0.4486369278373478


## Exécution de la simulation

In [28]:
Hermes.align(lambda_align, onduleur_BE)
Hermes.clear_impacts()

1

In [29]:
Hermes.generate(lambda_radiate)

5000

In [30]:
Hermes.radiate()

-1002871295

## Visualisations 

#### Fente de sortie du monochromateur 

In [31]:
fente.show_diagram(Hermes.active_chain[0].nrays)

          X         Y        dX        dY        Lambda
0  0.015077  0.000013  0.003372 -0.000065  6.000000e-09
1  0.018714  0.000077  0.004192 -0.000305  6.000000e-09
2  0.011668 -0.000007  0.002605  0.000048  6.000000e-09
3 -0.009415  0.000004 -0.002087  0.000025  6.000000e-09
4  0.005571  0.000003  0.001240 -0.000102  6.000000e-09


#### focalisation intermédiaire après M5 

In [32]:
foc_intermediaire.show_diagram(Hermes.active_chain[0].nrays)

          X         Y        dX        dY        Lambda
0  0.080264  0.000045  0.076600  0.000025  6.000000e-09
1  0.132677 -0.000128  0.117663 -0.000066  6.000000e-09
2  0.005213 -0.000106  0.007400 -0.000066  6.000000e-09
3  0.095775  0.000276  0.089006  0.000153  6.000000e-09
4  0.053931  0.000185  0.054788  0.000106  6.000000e-09


#### Plan du PEEM

In [33]:
peem.show_diagram(Hermes.active_chain[0].nrays)

          X         Y        dX        dY        Lambda
0 -0.761367  0.335927  0.096883  0.182450  6.000000e-09
1 -1.730293  0.333781  0.144405  0.180801  6.000000e-09
2  0.749168  0.334081  0.274417  0.175835  6.000000e-09
3 -1.004005  0.338783  0.112057  0.182916  6.000000e-09
4 -0.427935  0.337601  0.065750  0.183336  6.000000e-09
